In [1]:
import pandas as pd
import xml.etree.cElementTree as ET
from pathlib import Path

project_path = Path.cwd().parent
DATA_PATH = Path(project_path, 'apple_health_export', 'export.xml')
print(DATA_PATH) # We're using old data

/Users/frootloops/Documents/pyprojects/runnersdash/apple_health_export/export.xml


In [ ]:
# 
with open(DATA_PATH, 'r') as f:
    et_object = ET.parse(f)



In [ ]:
root.attrib